# Assignment 14: Sudoku

_This notebook is a derivative work of ["Sudoku"](https://colab.research.google.com/drive/1HXsCrHp8HqHJPF9RR1ZcfG7ehr_qP8e5) by [Luca de Alfaro](https://sites.google.com/a/ucsc.edu/luca/home), which is licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/), distributed by permission of the original author.  This notebook is licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by [Peter Alvaro](https://users.soe.ucsc.edu/~palvaro/) and [Lindsey Kuper](https://users.soe.ucsc.edu/~lkuper/)._

# Instructions

## Notebook Format

This is a homework notebook.  It consists of various types of cells: 

* Text cells: you should read them!
* Code cells: you should run them, as they may set up the problems that you are asked to solve.
* **Solution** cells: These are cells where you should enter a solution.  You will see a marker in these cells that indicates where your work should be inserted:  

```
    # YOUR CODE HERE
```    

* Test cells: These cells contain some tests, and are worth some points.  You should run the cells as a way to debug your code, and to see if you understood the question, and whether the output of your code is produced in the correct format.

**When we grade your notebook, we will run many additional tests in addition to the ones you see here.  (Otherwise, you'd be able to get full credit by hard-coding the desired output!)**

## Working on Your Notebook

To work on your notebook, we recommend using Colab.  Working in Colab has many benefits:

  * You don't have to maintain a working Python environment on your own machine; you can work from any machine that has an internet connection.
  * Colab preserves the revision history, which is useful for many reasons.
  * Your work is automatically saved in Google Drive.
  
In Colab, go to "File > Save a copy in Drive..." and then you'll have your own copy to work on.

## Submitting Your Notebook

Before you turn your finished notebook in, it's a good idea to sure everything runs as expected. First, **factory reset the runtime** in Colab (go to "Runtime > Factory reset runtime").  Then, **run all the cells** (go to "Runtime > Run all") in Colab.

(A word of caution: when you use "Run all", Colab tries to run cells in parallel, which can get you into trouble if later cells run before earlier ones on which they depend.  So, you might instead want to run all the cells sequentially yourself after resetting the runtime.  You can start at the top of the notebook, hold down the Shift key, and press Enter repeatedly to run one cell after another in order.)

Submit your work as follows: 

  * Download the notebook from Colab, clicking on "File > Download .ipynb".
  * Upload the resulting file to [this Google form](https://forms.gle/wj8QF8dcad2oxnLR6).
  * **Deadline: 9pm Wednesday, March 18.**

You can submit multiple times, and the last submission before the deadline will be used to assign you a grade.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and the names of anyone you collaborated with in the below cells.  For collaborators, list anyone who (for example) discussed the general techniques involved with you, or pointed you to useful Python documentation -- collaboration of that kind is encouraged.  Remember that sharing code or using others' code is not allowed.

In [ ]:
NAME = "Stephen Ip"
COLLABORATORS = ""

## What Happens Next? 

After you submit your notebook, your instructor at some point will retreat to a secret hideout, put on some good music, and run some mysterious scripts.  These will generate two things: 

  * Your grade, which goes into a spreadsheet. 
  * Feedback, shared to you as a PDF file on Google Drive.  The PDF shows your work, your grade, the tests that passed and those that failed, and any comments left by the instructor and the TAs.

# Testing

Make sure to run the following cell to make sure that the Python testing framework, `nose`, is installed.  (It's required for the rest of the notebook to work.)

In [ ]:
try:
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal, assert_greater_equal
except:
    !pip install nose
    from nose.tools import assert_equal, assert_almost_equal
    from nose.tools import assert_true, assert_false
    from nose.tools import assert_not_equal, assert_greater_equal

## Sudoku

During [lecture](https://colab.research.google.com/drive/1PtWLOBwItfCApTC0WdWxwjDD8RwJQUzq), we saw how by combining *constraint propagation* and *search*, we can implement an efficient solver for Sudoku puzzles.  Below, we reiterate the code we saw during lecture.

In [ ]:
def getel(s):
    """Returns the unique element in a singleton set (or list)."""
    assert len(s) == 1
    return list(s)[0]

In [ ]:
import json

class Sudoku:
    
    def __init__(self, elements):
        """The `elements` argument can be one of:
        Case 1: an instance of Sudoku.  In that case, we initialize an 
        object to be equal to (a copy of) the provided instance.
        Case 2: a list of 9 strings of length 9 each.
        Each string represents a row of the initial Sudoku puzzle,
        with either a digit 1..9 in it, or with a blank or _ to signify
        a blank cell.
        Case 3: a list of lists of sets, used to initialize the problem."""

        # Case 1
        if isinstance(elements, Sudoku):
            # We let self.m consist of copies of each set in elements.m
            self.m = [[x.copy() for x in row] for row in elements.m]

        # Cases 2 and 3
        else:
            assert len(elements) == 9
            for s in elements:
                assert len(s) == 9
            # We let self.m be our Sudoku problem, a 9x9 matrix of sets. 
            self.m = []
            for s in elements:
                row = []
                for c in s:
                    # Case 2
                    if isinstance(c, str):
                        if c.isdigit():
                            row.append({int(c)})
                        else:
                            row.append({1, 2, 3, 4, 5, 6, 7, 8, 9})
                    # Case 3
                    else:
                        assert isinstance(c, set)
                        row.append(c)
                self.m.append(row)               
            
    def show(self, details=False):
        """Prints out the Sudoku matrix.  If details=False, we print out
        the digits only for cells that have singleton sets (where only
        one digit can fit).  If details=True, for each cell, we display the 
        sets associated with the cell."""
        if details:
            print("+-----------------------------+-----------------------------+-----------------------------+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    # We represent the set {2, 3, 5} via _23_5____
                    s = ''
                    for k in range(1, 10):
                        s += str(k) if k in self.m[i][j] else '_'
                    r += s
                    r += '|' if (j + 1) % 3 == 0 else ' '                        
                print(r)
                if (i + 1) % 3 == 0:
                    print("+-----------------------------+-----------------------------+-----------------------------+")
        else:
            print("+---+---+---+")
            for i in range(9):
                r = '|'
                for j in range(9):
                    if len(self.m[i][j]) == 1:
                        r += str(getel(self.m[i][j]))
                    else:
                        r += "."
                    if (j + 1) % 3 == 0:
                        r += "|"
                print(r)
                if (i + 1) % 3 == 0:
                    print("+---+---+---+")
                    
                    
    def to_string(self):
        """This method is useful for producing a representation that 
        can be used in testing."""
        as_lists = [[list(self.m[i][j]) for j in range(9)] for i in range(9)]
        return json.dumps(as_lists)
    
    @staticmethod
    def from_string(s):
        """Inverse of above."""
        as_lists = json.loads(s)
        as_sets = [[set(el) for el in row] for row in as_lists]
        return Sudoku(as_sets)  
    
    def __eq__(self, other):
        """Useful for testing."""
        return self.m == other.m

Let us input a problem (the Sudoku example found on [this Wikipedia page](https://en.wikipedia.org/wiki/Sudoku)) and check that our serialization and deserialization works.

In [ ]:
sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show()
sd.show(details=True)

+---+---+---+
|53.|.7.|...|
|6..|195|...|
|.98|...|.6.|
+---+---+---+
|8..|.6.|..3|
|4..|8.3|..1|
|7..|.2.|..6|
+---+---+---+
|.6.|...|28.|
|...|419|..5|
|...|.8.|.79|
+---+---+---+
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 123456789|123456789 ______7__ 123456789|123456789 123456789 123456789|
|_____6___ 123456789 123456789|1________ ________9 ____5____|123456789 123456789 123456789|
|123456789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+---------------------

In [ ]:
s = sd.to_string()
sdd = Sudoku.from_string(s)
sdd.show(details=True)
assert_equal(sd, sdd)

+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 123456789|123456789 ______7__ 123456789|123456789 123456789 123456789|
|_____6___ 123456789 123456789|1________ ________9 ____5____|123456789 123456789 123456789|
|123456789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|123456789 _____6___ 123456789|123456789 123456789 123456789|_2_______ _______8_ 123456789|
|123456789 123456789 123456789|___4_____ 1________ ________9|123456789 123456789

### Problem 1: Completing the definition of `propagate_cell` (20 points)

When the set in a Sudoku cell contains only one element, this means that the digit at that cell is known.
 
We can then propagate the knowledge, ruling out that digit in the same row, in the same column, and in the same 3x3 block.

For this problem, you will write a method that propagates the constraint from a single cell.  The method will return the list of newly-determined cells, that is, the list of cells who also now (but not before) are associated with a 1-element set.  This is useful, because we can then propagate the constraints from those cells in turn.  Further, if an empty set is ever generated, we raise the exception `Unsolvable`: this means that there is no solution to the proposed Sudoku puzzle. 

First, we implement a `ruleout` method for the `Sudoku` class.  The `ruleout` method takes a cell (in terms of its indices `i` and `j`) and a value `x`, and rules out the value `x` as a possibility for that cell.  If this makes the set of possibilities for the cell empty, then `ruleout` will raise an `Unsolvable` exception.

If ruling out `x` narrows the possibilities for the specified cell to a single possibility, `ruleout` returns the set `{(i, j)}`.  Otherwise, it returns the empty set.

In [ ]:
class Unsolvable(Exception):
    pass

def sudoku_ruleout(self, i, j, x):
    """Takes as input a cell (i, j), and a value x.
    Removes x from the set of possibilities for that cell, if present, and:
    - if the resulting set is empty, raises Unsolvable;
    - if the cell used to be a non-singleton cell and is now a singleton 
      cell, then returns the set {(i, j)};
    - otherwise, returns the empty set."""
    c = self.m[i][j]
    n = len(c)
    c.discard(x)
    self.m[i][j] = c
    if len(c) == 0:
        raise Unsolvable()
    return {(i, j)} if 1 == len(c) < n else set()    

Sudoku.ruleout = sudoku_ruleout

Now we can implement the `propagate_cell` method.  `propagate_cell` takes a two-element tuple `ij` of cell indices, and it propagates the singleton value at cell `(i, j)`, returning the set of newly-singleton cells that result from propagation.

The provided code below is partly written for you.  It checks that `(i, j)` is a singleton cell, it propagates singleton values to cells in the same **row**.and it keeps track of the newly-singleton cells and returns a list of them.  

Your job is to edit the below definition of `propagate_cell` in the designated places to implement propagation singleton values to cells in the same **column** and in the same **3x3 block**.

As an example, suppose you have the following puzzle:

In [ ]:
sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.show(details=True)

+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 123456789|123456789 ______7__ 123456789|123456789 123456789 123456789|
|_____6___ 123456789 123456789|1________ ________9 ____5____|123456789 123456789 123456789|
|123456789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|123456789 _____6___ 123456789|123456789 123456789 123456789|_2_______ _______8_ 123456789|
|123456789 123456789 123456789|___4_____ 1________ ________9|123456789 123456789

Suppose you propagate the value of the singleton cell in the top left (cell `(0,0)`), whose value is 5, by calling `sd.propagate_cell((0, 0))` using the below provided implementation of `propagate_cell`.  Calling `sd.show(details=True)` would then show a puzzle where 5 has been ruled out as a possibility in every other cell in the same row:

```
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 1234_6789|1234_6789 ______7__ 1234_6789|1234_6789 1234_6789 1234_6789|
|_____6___ 123456789 123456789|1________ ________9 ____5____|123456789 123456789 123456789|
|123456789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|123456789 _____6___ 123456789|123456789 123456789 123456789|_2_______ _______8_ 123456789|
|123456789 123456789 123456789|___4_____ 1________ ________9|123456789 123456789 ____5____|
|123456789 123456789 123456789|123456789 _______8_ 123456789|123456789 ______7__ ________9|
+-----------------------------+-----------------------------+-----------------------------+
```

But if the `propagate_cell` method additionally propagated to columns and 3x3 blocks, you would get a puzzle where 5 has been ruled out as a possibility in every other cell in the same row, column, and 3x3 block, as shown below:

```
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 1234_6789|1234_6789 ______7__ 1234_6789|1234_6789 1234_6789 1234_6789|
|_____6___ 1234_6789 1234_6789|1________ ________9 ____5____|123456789 123456789 123456789|
|1234_6789 ________9 _______8_|123456789 123456789 123456789|123456789 _____6___ 123456789|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 123456789 123456789|123456789 _____6___ 123456789|123456789 123456789 __3______|
|___4_____ 123456789 123456789|_______8_ 123456789 __3______|123456789 123456789 1________|
|______7__ 123456789 123456789|123456789 _2_______ 123456789|123456789 123456789 _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1234_6789 _____6___ 123456789|123456789 123456789 123456789|_2_______ _______8_ 123456789|
|1234_6789 123456789 123456789|___4_____ 1________ ________9|123456789 123456789 ____5____|
|1234_6789 123456789 123456789|123456789 _______8_ 123456789|123456789 ______7__ ________9|
+-----------------------------+-----------------------------+-----------------------------+
```

That is the behavior we are asking you to implement.

**Hint:** You can look at the way that propagation is implemented for rows, and use that as a guide for how to implement it for columns.  Propagation to 3x3 blocks is slightly trickier because you have to work out which cells to propagate to.

In [ ]:
def sudoku_propagate_cell(self, ij):
    """Takes a two-element tuple `ij` of cell indices.
    Propagates the singleton value at cell (i, j), returning the set 
    of newly-singleton cells."""
    i, j = ij
    if len(self.m[i][j]) > 1:
        # Nothing to propagate from cell (i,j).
        return set()
    # We keep track of the newly-singleton cells.
    newly_singleton = set()
    x = getel(self.m[i][j]) # Value at (i, j). 

    # Same row.
    for jj in range(9):
        if jj != j: # Do not propagate to the element itself.
            newly_singleton.update(self.ruleout(i, jj, x))
            
    # Same column.
    for ii in range(9):
        if ii != i:
            newly_singleton.update(self.ruleout(ii, j, x))
    
    # Same 3x3 block of cells.
    if i <= 2:
        row = 'top'
    elif i <= 5:
        row = 'mid'
    elif i <= 8:
        row = 'bot'
    if j <= 2:
        column = 'left'
    elif j <= 5:
        column = 'mid'
    elif j <= 8:
        column = 'right'
    
    if row == 'top':
        if column == 'left':
            for ii in range(3):
                for jj in range(3):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
        elif column == 'mid':
            for ii in range(3):
                for jj in range(3, 6):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
        elif column == 'right':
            for ii in range(3):
                for jj in range(6, 9):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
    elif row == 'mid':
        if column == 'left':
            for ii in range(3, 6):
                for jj in range(3):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
        elif column == 'mid':
            for ii in range(3, 6):
                for jj in range(3, 6):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
        elif column == 'right':
            for ii in range(3, 6):
                for jj in range(6, 9):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
    elif row == 'bot':
        if column == 'left':
            for ii in range(6, 9):
                for jj in range(3):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
        elif column == 'mid':
            for ii in range(6, 9):
                for jj in range(3, 6):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))
        elif column == 'right':
            for ii in range(6, 9):
                for jj in range(6, 9):
                    if ii != i and jj != j:
                        newly_singleton.update(self.ruleout(ii, jj, x))


    # Returns the list of newly-singleton cells.
    return newly_singleton

Sudoku.propagate_cell = sudoku_propagate_cell

In [ ]:
### Feel free to use this cell to write your own tests.
### You will not be graded on what you write here.


In [ ]:
### Propagating to the same column

tsd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])

# Ensure that the value 5 in cell (0, 0) has been ruled out for other cells in the same column
tsd.propagate_cell((0,0))
assert_equal(
    [tsd.m[i][0] for i in range(9)],
    [{5},
     {6},
     {1, 2, 3, 4, 6, 7, 8, 9},
     {8},
     {4},
     {7},
     {1, 2, 3, 4, 6, 7, 8, 9},
     {1, 2, 3, 4, 6, 7, 8, 9},
     {1, 2, 3, 4, 6, 7, 8, 9}])

tsd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])

# Ensure that the value 3 in cell (4, 5) has been ruled out for other cells in the same column
tsd.propagate_cell((4,5))
assert_equal(
    [tsd.m[i][5] for i in range(9)],
    [{1, 2, 4, 5, 6, 7, 8, 9},
     {5},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {3},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {9},
     {1, 2, 4, 5, 6, 7, 8, 9}])

tsd = Sudoku([
    '6____894_',
    '9____61__',
    '_7__4____',
    '2__61____',
    '______2__',
    '_89__2___',
    '____6___5',
    '_______3_',
    '8____16__'
])

# Ensure that the value 3 in cell (7, 7) has been ruled out for other cells in the same column
tsd.propagate_cell((7, 7))
assert_equal(
    [tsd.m[i][7] for i in range(9)],
    [{4},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {1, 2, 4, 5, 6, 7, 8, 9},
     {3},
     {1, 2, 4, 5, 6, 7, 8, 9}])


In [ ]:
### Propagating to the same 3x3 block

tsd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])

# Ensure that the value 5 in cell (0, 0) has been ruled out for other cells in the same 3x3 block
tsd.propagate_cell((0,0))
assert_equal(
    [tsd.m[i][j] for i in range(3) for j in range(3)],
    [{5},                          {3},                      {1, 2, 3, 4, 6, 7, 8, 9},
     {6},                          {1, 2, 3, 4, 6, 7, 8, 9}, {1, 2, 3, 4, 6, 7, 8, 9},
     {1, 2, 3, 4, 6, 7, 8, 9},     {9},                      {8}])

tsd = Sudoku([
    '6____894_',
    '9____61__',
    '_7__4____',
    '2__61____',
    '______2__',
    '_89__2___',
    '____6___5',
    '_______3_',
    '8____16__'
])

# Ensure that the value 9 in cell (0, 6) has been ruled out for other cells in the same 3x3 block
tsd.propagate_cell((0, 6))
assert_equal(
    [tsd.m[i][j] for i in range(3) for j in range(6, 9)],
    [{9},                          {4},                          {1, 2, 3, 4, 5, 6, 7, 8},
     {1},                          {1, 2, 3, 4, 5, 6, 7, 8},     {1, 2, 3, 4, 5, 6, 7, 8},
     {1, 2, 3, 4, 5, 6, 7, 8},     {1, 2, 3, 4, 5, 6, 7, 8},     {1, 2, 3, 4, 5, 6, 7, 8}])

tsd = Sudoku([
    '6____894_',
    '9____61__',
    '_7__4____',
    '2__61____',
    '______2__',
    '_89__2___',
    '____6___5',
    '_______3_',
    '8____16__'
])

# Ensure that the value 3 in cell (7, 7) has been ruled out for other cells in the same 3x3 block
tsd.propagate_cell((7, 7))
assert_equal(
    [tsd.m[i][j] for i in range(6, 9) for j in range(6, 9)],
    [{1, 2, 4, 5, 6, 7, 8, 9},     {1, 2, 4, 5, 6, 7, 8, 9},     {5},
     {1, 2, 4, 5, 6, 7, 8, 9},     {3},                          {1, 2, 4, 5, 6, 7, 8, 9},
     {6},                          {1, 2, 4, 5, 6, 7, 8, 9},     {1, 2, 4, 5, 6, 7, 8, 9}])


In [ ]:
### More tests for cell propagation

tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)
try:
    tsd.propagate_cell((0, 2))
except Unsolvable:
    print("Good! It was unsolvable.")
else:
    raise Exception("Hey, it was unsolvable")
    
tsd = Sudoku.from_string('[[[5], [3], [2], [6], [7], [8], [9], [1, 2, 4], [2, 3]], [[6], [7], [1, 2, 4, 7], [1, 2, 3], [9], [5], [3], [1, 2, 4], [8]], [[1, 2], [9], [8], [3], [4], [1, 2], [5], [6], [7]], [[8], [5], [9], [1, 9, 7], [6], [1, 4, 9, 7], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[1, 9], [6], [1, 5, 9, 7], [9, 5, 7], [3], [9, 7], [2], [8], [4]], [[9, 2], [8], [9, 2, 7], [4], [1], [9, 2, 7], [6], [3], [5]], [[3], [4], [2, 3, 4, 5], [2, 5, 6], [8], [6], [1], [7], [9]]]')
tsd.show(details=True)
assert_equal(tsd.propagate_cell((0, 2)), {(0, 8), (2, 0)})


+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ _2_______|_____6___ ______7__ _______8_|________9 12_4_____ _2_______|
|_____6___ ______7__ 12_4__7__|123______ ________9 ____5____|__3______ 12_4_____ _______8_|
|12_______ ________9 _______8_|__3______ ___4_____ 12_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|1_____7_9 _____6___ 1__4__7_9|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_______9 _____6___ 1___5_7_9|____5_7_9 __3______ ______7_9|_2_______ _______8_ ___4_____|
|_2______9 _______8_ _2____7_9|___4_____ 1________ _2____7_9|_____6___ __3______

### Problem 2: Propagating all cells, repeatedly (20 points)

During lecture, we considered what happens if we propagate all singleton values once:

In [ ]:
def sudoku_propagate_all_cells_once(self):
    """This function propagates the constraints from all singletons."""
    for i in range(9):
        for j in range(9):
            self.propagate_cell((i, j))
            
Sudoku.propagate_all_cells_once = sudoku_propagate_all_cells_once

In [ ]:
tsd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.propagate_all_cells_once()
sd.show(details=True)

+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ 12_4_____|_2___6___ ______7__ _2_4_6_8_|1__4___89 12_4____9 _2_4___8_|
|_____6___ _2_4__7__ _2_4__7__|1________ ________9 ____5____|__34__78_ _234_____ _2_4__78_|
|12_______ ________9 _______8_|_23______ __34_____ _2_4_____|1_345_7__ _____6___ _2_4__7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ 12__5____ 12__5___9|____5_7_9 _____6___ 1__4__7__|___45_7_9 _2_45___9 __3______|
|___4_____ _2__5____ _2__56__9|_______8_ ____5____ __3______|____5_7_9 _2__5___9 1________|
|______7__ 1___5____ 1_3_5___9|____5___9 _2_______ 1__4_____|___45__89 ___45___9 _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|1_3_____9 _____6___ 1_345_7_9|__3_5_7__ __3_5____ ______7__|_2_______ _______8_ ___4_____|
|_23______ _2____78_ _23___7__|___4_____ 1________ ________9|__3__6___ __3______

We see that propagating once leaves us with some new singleton cells, such as `{5}` for the middle cell in the above puzzle.  (If you haven't yet correctly implemented problem 1, the middle cell in the above puzzle may not be a singleton cell yet.  If it isn't, don't move on to this problem until problem 1 is done.)  We need to make sure that these new singleton values are propagated. 

For this problem, you will complete the implementation of a method `full_propagation` that starts with a set of `to_propagate` cells (if it is not specified, then we just take it to consist of all singleton cells; this is already implemented for you).

The `full_propagation` method should pick a cell from the `to_propagate` set, and propagate from it, adding any newly singleton cell to `to_propagate`.  It continues until there are no more singleton cells to be propagated.  `full_propagation` does not return a value.  This can be implemented in about 3 lines of code.

Here's what the above example puzzle will look like if we do full propagation correctly:

```
+-----------------------------+-----------------------------+-----------------------------+
|____5____ __3______ ___4_____|_____6___ ______7__ _______8_|________9 1________ _2_______|
|_____6___ ______7__ _2_______|1________ ________9 ____5____|__3______ ___4_____ _______8_|
|1________ ________9 _______8_|__3______ ___4_____ _2_______|____5____ _____6___ ______7__|
+-----------------------------+-----------------------------+-----------------------------+
|_______8_ ____5____ ________9|______7__ _____6___ 1________|___4_____ _2_______ __3______|
|___4_____ _2_______ _____6___|_______8_ ____5____ __3______|______7__ ________9 1________|
|______7__ 1________ __3______|________9 _2_______ ___4_____|_______8_ ____5____ _____6___|
+-----------------------------+-----------------------------+-----------------------------+
|________9 _____6___ 1________|____5____ __3______ ______7__|_2_______ _______8_ ___4_____|
|_2_______ _______8_ ______7__|___4_____ 1________ ________9|_____6___ __3______ ____5____|
|__3______ ___4_____ ____5____|_2_______ _______8_ _____6___|1________ ______7__ ________9|
+-----------------------------+-----------------------------+-----------------------------+
```

As you can see, the puzzle can be completely solved using constraint propagation.

In [ ]:
def sudoku_full_propagation(self, to_propagate=None):
    """Iteratively propagates from all singleton cells, and from all 
    newly discovered singleton cells, until no more propagation is possible."""
    if to_propagate is None:
        to_propagate = {(i, j) for i in range(9) for j in range(9)}
    while len(to_propagate) != 0:
        ij = to_propagate.pop()
        to_propagate.update(self.propagate_cell(ij))

    
Sudoku.full_propagation = sudoku_full_propagation

In [ ]:
### Feel free to use this cell to write your own tests.
### You will not be graded on what you write here.


In [ ]:
### Tests for `full_propagation`

sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.full_propagation()
sd.show()

# Spot checks for a few cells
assert_equal(sd.m[0][2], {4})
assert_equal(sd.m[1][1], {7})
assert_equal(sd.m[8][1], {4})
assert_equal(sd.m[4][7], {9})


+---+---+---+
|534|678|912|
|672|195|348|
|198|342|567|
+---+---+---+
|859|761|423|
|426|853|791|
|713|924|856|
+---+---+---+
|961|537|284|
|287|419|635|
|345|286|179|
+---+---+---+


In [ ]:
### More tests for `full_propagation`

# Full propagation is enough to solve this puzzle...
sd = Sudoku([
    '53__7____',
    '6__195___',
    '_98____6_',
    '8___6___3',
    '4__8_3__1',
    '7___2___6',
    '_6____28_',
    '___419__5',
    '____8__79'
])
sd.full_propagation()
sd.show()
sdd = Sudoku.from_string('[[[5], [3], [4], [6], [7], [8], [9], [1], [2]], [[6], [7], [2], [1], [9], [5], [3], [4], [8]], [[1], [9], [8], [3], [4], [2], [5], [6], [7]], [[8], [5], [9], [7], [6], [1], [4], [2], [3]], [[4], [2], [6], [8], [5], [3], [7], [9], [1]], [[7], [1], [3], [9], [2], [4], [8], [5], [6]], [[9], [6], [1], [5], [3], [7], [2], [8], [4]], [[2], [8], [7], [4], [1], [9], [6], [3], [5]], [[3], [4], [5], [2], [8], [6], [1], [7], [9]]]')
assert_equal(sd, sdd)

# ...but not this one 
sd = Sudoku([
    '8________',
    '__36_____',
    '_7__9_2__',
    '_5___7___',
    '____457__',
    '___1___3_',
    '__1____68',
    '__85___1_',
    '_9____4__'
])
sd.full_propagation()
sd.show()
sdd = Sudoku.from_string('[[[8], [1, 2, 4, 6], [2, 4, 5, 6, 9], [2, 3, 4, 7], [1, 2, 3, 5, 7], [1, 2, 3, 4], [1, 3, 5, 6, 9], [4, 5, 7, 9], [1, 3, 4, 5, 6, 7, 9]], [[1, 2, 4, 5, 9], [1, 2, 4], [3], [6], [1, 2, 5, 7, 8], [1, 2, 4, 8], [1, 5, 8, 9], [4, 5, 7, 8, 9], [1, 4, 5, 7, 9]], [[1, 4, 5, 6], [7], [4, 5, 6], [3, 4, 8], [9], [1, 3, 4, 8], [2], [4, 5, 8], [1, 3, 4, 5, 6]], [[1, 2, 3, 4, 6, 9], [5], [2, 4, 6, 9], [2, 3, 8, 9], [2, 3, 6, 8], [7], [1, 6, 8, 9], [2, 4, 8, 9], [1, 2, 4, 6, 9]], [[1, 2, 3, 6, 9], [1, 2, 3, 6, 8], [2, 6, 9], [2, 3, 8, 9], [4], [5], [7], [2, 8, 9], [1, 2, 6, 9]], [[2, 4, 6, 7, 9], [2, 4, 6, 8], [2, 4, 6, 7, 9], [1], [2, 6, 8], [2, 6, 8, 9], [5, 6, 8, 9], [3], [2, 4, 5, 6, 9]], [[2, 3, 4, 5, 7], [2, 3, 4], [1], [2, 3, 4, 7, 9], [2, 3, 7], [2, 3, 4, 9], [3, 5, 9], [6], [8]], [[2, 3, 4, 6, 7], [2, 3, 4, 6], [8], [5], [2, 3, 6, 7], [2, 3, 4, 6, 9], [3, 9], [1], [2, 3, 7, 9]], [[2, 3, 5, 6, 7], [9], [2, 5, 6, 7], [2, 3, 7, 8], [1, 2, 3, 6, 7, 8], [1, 2, 3, 6, 8], [4], [2, 5, 7], [2, 3, 5, 7]]]')
assert_equal(sd, sdd)


+---+---+---+
|534|678|912|
|672|195|348|
|198|342|567|
+---+---+---+
|859|761|423|
|426|853|791|
|713|924|856|
+---+---+---+
|961|537|284|
|287|419|635|
|345|286|179|
+---+---+---+
+---+---+---+
|8..|...|...|
|..3|6..|...|
|.7.|.9.|2..|
+---+---+---+
|.5.|..7|...|
|...|.45|7..|
|...|1..|.3.|
+---+---+---+
|..1|...|.68|
|..8|5..|.1.|
|.9.|...|4..|
+---+---+---+


For the above puzzle that can't be solved with propagation alone, you can solve it using the search algorithm shown in lecture -- feel free to try it on your own!